<a href="https://colab.research.google.com/github/fred-ykv/Value-Investing-In-Python/blob/master/Web_Scrap_Finviz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen

pd.set_option('display.max_colwidth', 25)

# Entrada do Ticker

In [2]:
symbol = input('Enter a ticker: ')
print ('Getting data for ' + symbol + '...\n')

Enter a ticker: MLI
Getting data for MLI...



# Set up scraper

In [3]:
url = ("http://finviz.com/quote.ashx?t=" + symbol.lower())
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
html = soup(webpage, "html.parser")

In [4]:
def get_fundamentals():
    try:
        # Find fundamentals table
        fundamentals = pd.read_html(str(html), attrs = {'class': 'snapshot-table2'})[0]
        
        # Clean up fundamentals dataframe
        fundamentals.columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']
        colOne = []
        colLength = len(fundamentals)
        for k in np.arange(0, colLength, 2):
            colOne.append(fundamentals[f'{k}'])
        attrs = pd.concat(colOne, ignore_index=True)
    
        colTwo = []
        colLength = len(fundamentals)
        for k in np.arange(1, colLength, 2):
            colTwo.append(fundamentals[f'{k}'])
        vals = pd.concat(colTwo, ignore_index=True)
        
        fundamentals = pd.DataFrame()
        fundamentals['Attributes'] = attrs
        fundamentals['Values'] = vals
        fundamentals = fundamentals.set_index('Attributes')
        return fundamentals

    except Exception as e:
        return e

In [5]:
def get_news():
    try:
        # Find news table
        news = pd.read_html(str(html), attrs = {'class': 'fullview-news-outer'})[0]
        links = []
        for a in html.find_all('a', class_="tab-link-news"):
            links.append(a['href'])
        
        # Clean up news dataframe
        news.columns = ['Date', 'News Headline']
        news['Article Link'] = links
        news = news.set_index('Date')
        return news

    except Exception as e:
        return e

In [6]:
def get_insider():
    try:
        # Find insider table
        insider = pd.read_html(str(html), attrs = {'class': 'body-table'})[0]
        
        # Clean up insider dataframe
        insider = insider.iloc[1:]
        insider.columns = ['Trader', 'Relationship', 'Date', 'Transaction', 'Cost', '# Shares', 'Value ($)', '# Shares Total', 'SEC Form 4']
        insider = insider[['Date', 'Trader', 'Relationship', 'Transaction', 'Cost', '# Shares', 'Value ($)', '# Shares Total', 'SEC Form 4']]
        insider = insider.set_index('Date')
        return insider

    except Exception as e:
        return e

In [7]:
print ('Fundamental Ratios: ')
print(get_fundamentals())

Fundamental Ratios: 
                 Values
Attributes             
Index                 -
Market Cap        4.16B
Income          658.30M
Sales             3.98B
Book/sh           29.24
...                 ...
ATR                2.12
Volatility  2.63% 3.23%
Prev Close        72.92
Price             74.02
Change            1.51%

[72 rows x 1 columns]


In [8]:
print ('\nRecent News: ')
print(get_news())


Recent News: 
Length of values (100) does not match length of index (103)


In [9]:
print ('\nRecent Insider Trades: ')
print(get_insider())


Recent Insider Trades: 
                          Trader              Relationship      Transaction  \
Date                                                                          
Feb 09             HANSEN JOHN B                  Director             Sale   
Feb 07    Christopher Gregory L.  Chairman of the Board...             Sale   
Dec 20  Miritello Christopher...  VP, General Counsel, ...  Option Exercise   
Nov 23             HANSEN JOHN B                  Director             Sale   
Oct 20             HANSEN JOHN B                  Director             Sale   
Oct 20             HANSEN JOHN B                  Director             Sale   
Aug 25             HANSEN JOHN B                  Director             Sale   
Aug 09             HANSEN JOHN B                  Director             Sale   
Jul 26             HANSEN JOHN B                  Director             Sale   
Apr 21             HANSEN JOHN B                  Director             Sale   
Apr 12          GLADSTEIN G

In [10]:
print ('\nFundamental Ratios: ')
print(get_fundamentals())

print ('\nRecent News: ')
print(get_news())

print ('\nRecent Insider Trades: ')
print(get_insider())


Fundamental Ratios: 
                 Values
Attributes             
Index                 -
Market Cap        4.16B
Income          658.30M
Sales             3.98B
Book/sh           29.24
...                 ...
ATR                2.12
Volatility  2.63% 3.23%
Prev Close        72.92
Price             74.02
Change            1.51%

[72 rows x 1 columns]

Recent News: 
Length of values (100) does not match length of index (103)

Recent Insider Trades: 
                          Trader              Relationship      Transaction  \
Date                                                                          
Feb 09             HANSEN JOHN B                  Director             Sale   
Feb 07    Christopher Gregory L.  Chairman of the Board...             Sale   
Dec 20  Miritello Christopher...  VP, General Counsel, ...  Option Exercise   
Nov 23             HANSEN JOHN B                  Director             Sale   
Oct 20             HANSEN JOHN B                  Director           

In [11]:
print ('Fundamental Ratios: ')
fundamentals = get_fundamentals()
if 'Market Cap' in fundamentals.index:
    print(fundamentals.loc['Market Cap'])
else:
    print('Market Cap is not listed.')

Fundamental Ratios: 
Values    4.16B
Name: Market Cap, dtype: object


In [12]:
market_cap_str = fundamentals.loc['Market Cap']['Values']
market_cap_str = market_cap_str.strip()

if market_cap_str[-1] == 'B':
    market_cap = float(market_cap_str[:-1]) * 1000000000
elif market_cap_str[-1] == 'M':
    market_cap = float(market_cap_str[:-1]) * 1000000
else:
    market_cap = float(market_cap_str)

print(f'Market Cap: {int(market_cap)}')

Market Cap: 4160000000


In [13]:
roi_str = fundamentals.loc['ROI']['Values']
roi_str = roi_str.replace('%', '')
roi = float(roi_str)
roi_perc = roi/100
roi_perc_str = str(roi_perc).replace('.', ',')

print(f'ROI: {roi_perc_str}')

ROI: 0,4


In [14]:
roe_str = fundamentals.loc['ROE']['Values']
roe_str = roe_str.replace('%', '')
roe = float(roe_str)
roe_perc = roe/100
roe_perc_str = str(roe_perc).replace('.', ',')

print(f'ROE: {roe_perc_str}')

ROE: 0,462


In [15]:
payout_str = fundamentals.loc['Payout']['Values']
payout_str = payout_str.replace('%', '')
payout = float(payout_str)
payout_perc = payout/100
payout_perc_str = str(payout_perc).replace('.', ',')

print(f'Payout: {payout_perc_str}')

Payout: 0,064


In [16]:
div_str = fundamentals.loc['Dividend %']['Values']
div_str = div_str.replace('%', '')
div = float(div_str)
div_perc = div/100
div_perc_str = str(div_perc).replace('.', ',')

print(f'Dividend %: {div_perc_str}')

Dividend %: 0,0165


In [17]:
epsn5_str = fundamentals.loc['EPS next 5Y']['Values']
epsn5_str = epsn5_str.replace('%', '')
epsn5 = float(epsn5_str)
epsn5_perc = epsn5/100
epsn5_perc_str = str(epsn5_perc).replace('.', ',')

print(f'EPS next 5: {epsn5_perc_str}')

EPS next 5: 0,12


In [18]:
eps_str = fundamentals.loc['EPS (ttm)']['Values']
eps = float(eps_str)
eps_str = str(eps).replace('.', ',')

print(f'EPS: {eps_str}')

EPS: 11,64


In [19]:
LPA_str = fundamentals.loc['Book/sh']['Values']
LPA = float(LPA_str)
LPA_str = str(LPA).replace('.', ',')

print(f'LPA: {LPA}')


LPA: 29.24


In [20]:
import math

eps = float(eps)
LPA = float(LPA)

# Calcular o valor intrínseco em dólar
valor_intrinseco = math.sqrt(eps * LPA * 22.5)

print(f"O valor intrínseco da ação é de ${valor_intrinseco:.2f}")


O valor intrínseco da ação é de $87.51


In [21]:
 price_str = fundamentals.loc['Price']['Values']
 price = float(price_str)
 price_str = str(price).replace('.', ',')

print(f'Preço Atual: {price_str}')

Preço Atual: 74,02


In [22]:
# Calcular a diferença entre o valor intrínseco e o preço atual
diferenca = valor_intrinseco - price

# Calcular a diferença percentual
dif_percentual = diferenca / valor_intrinseco * 100

# Verificar se está com ágio ou deságio
if price > valor_intrinseco:
    agio_desagio = 'ágio'
else:
    agio_desagio = 'deságio'

# Exibir os resultados
print(f"Valor intrínseco: ${valor_intrinseco:.2f}")
print(f"Preço atual: ${price:.2f}")
print(f"Diferença: ${diferenca:.2f}")
print(f"Diferença percentual: {dif_percentual:.2f}% {agio_desagio}")

if agio_desagio == 'ágio':
    recomendacao = 'Recomendação: Venda'
else:
    recomendacao = 'Recomendação: Compra'

print(recomendacao)

Valor intrínseco: $87.51
Preço atual: $74.02
Diferença: $13.49
Diferença percentual: 15.42% deságio
Recomendação: Compra


In [23]:
beta_str = fundamentals.loc['Beta']['Values']
beta = float(beta_str)
beta_str = str(beta).replace('.', ',')

print(f'Beta: {beta_str}')

Beta: 1,12


In [24]:
tax_reinvest_ll = 1 - payout_perc
tax_reinvest_ll_str = str(tax_reinvest_ll).replace('.', ',')

print(f'Taxa de Reinvestimento LL: {tax_reinvest_ll_str}')

Taxa de Reinvestimento LL: 0,9359999999999999


In [25]:
roe_str = fundamentals.loc['ROE']['Values']
roe_str = roe_str.replace('%', '')
roe = float(roe_str)
roe_perc = roe/100

gll = tax_reinvest_ll*roe_perc
gll_str = str(gll).replace('.', ',')

print(f'gLL: {gll_str}')

gLL: 0,432432


In [26]:
gain_leverage = roe_perc - roi_perc
gain_leverage_str = str(gain_leverage).replace('.', ',')

print(f'Ganho de Alavancagem: {gain_leverage_str}')

Ganho de Alavancagem: 0,062


In [27]:
gaf = roe_perc/roi_perc
gaf_str = str(gaf).replace('.', ',')

print(f'Grau de Alavancagem Financeira: {gaf_str}')

Grau de Alavancagem Financeira: 1,155


In [28]:
# Requisição Url
url = "https://ycharts.com/indicators/10_year_treasury_rate"
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

# Parser
html = soup(webpage, "html.parser")

# Buscando Valor
xpath = '/html/body/main/div/div[2]/div/div/div[2]'
value_elem = html.find('div', {'class':'key-stat-title'}).get_text()

if value_elem is not None:
    value = value_elem.strip()
    value = value.split(' ')[0] # Removendo Strings
    value = value.replace('%', '')

    # Removendo Caracteres
    Rf = float(value.replace(',', '.'))
    Rf_str = str(Rf).replace('.', ',') + '%' # Corrigindo Erro

    print(f'Taxa de Juros Livre de Risco (Rf): {Rf_str}')

Taxa de Juros Livre de Risco (Rf): 3,95%


In [29]:
# Requisição Url
url = "https://ycharts.com/indicators/us_consumer_price_index_yoy"
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

# Parser
html = soup(webpage, "html.parser")

# Buscando Valor
xpath = '/html/body/main/div/div[2]/div/div/div[2]'
value_elem = html.find('div', {'class':'key-stat-title'}).get_text()

if value_elem is not None:
    value = value_elem.strip()
    value = value.split(' ')[0] # Removendo Strings
    value = value.replace('%', '')

    # Removendo Caracteres
    inflation = float(value.replace(',', '.'))
    inflation_str = str(inflation).replace('.', ',') + '%' # Corrigindo Erro

    print(f'Taxa de Inflação Americana:: {inflation_str}')

Taxa de Inflação Americana:: 6,41%


In [30]:
url = 'https://ycharts.com/indicators/us_10year_government_bond_interest_rate'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
html = soup(webpage, "html.parser")

xpath = '/html/body/main/div/div[2]/div/div/div[2]'
value_elem = html.find('div', {'class':'key-stat-title'}).get_text()

if value_elem is not None:
    value = value_elem.strip()
    value = value.split(' ')[0] # Removendo Strings
    value = value.replace('%', '')

    # Removendo Caracteres
    interest_rate = float(value.replace(',', '.'))
    interest_rate_str = str(interest_rate).replace('.', ',') + '%' # Corrigindo Erro

    print(f'Taxa de Juros Livre de Risco (Rf): {interest_rate_str}')

Taxa de Juros Livre de Risco (Rf): 3,53%


In [31]:
# Set up scraper
url = 'https://www.investing.com/rates-bonds/brazil-1-year-bond-yield'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
html = soup(webpage, "html.parser")

rm_str = html.find('span', {'class': 'arial_26 inlineblock pid-24024-last'})
rm = float(rm_str.text)
rm_perc = rm/100
rm_perc_str = str(rm_perc).replace('.', ',')

print(f'Retorno Carteira de Mercado (Rm): {rm_perc_str}')

Retorno Carteira de Mercado (Rm): 0,13608


In [32]:
# Set up scraper
url = 'https://br.investing.com/economic-calendar/brazilian-cpi-410'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
html = soup(webpage, "html.parser")

inflation_str = html.find('div', {'class': 'arial_14 redFont'})
inflation = float(inflation_str.text.replace(',', '.').replace('%', ''))
inflation_perc_str = str(inflation).replace('.', ',')

print(f'Taxa de Inflação Brasil (%) : {inflation_perc_str}%')

Taxa de Inflação Brasil (%) : 5,77%


# **Balanço Patrimonial**

In [33]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.zacks.com/stock/quote/'+ symbol +'/balance-sheet'
user_agent = {'User-agent': 'Mozilla/5.0'}

# Obtendo a página
page = requests.get(url, headers = user_agent)

# Criando o objeto BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

# Extraindo os valores da tabela
table_rows = soup.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    print(row)

[]
['Strong Buy', '24.28%']
['Buy', '17.83%']
['Hold', '9.19%']
['Sell', '5.00%']
['Strong Sell', '2.16%']
['500', '10.51%']
['A']
['A']
['A']
['A']
[]
['Assets', '\xa0', '\xa0', '\xa0', '\xa0', '\xa0']
['Cash & Equivalents', '679', '88', '119', '98', '73']
['Receivables', '380', '472', '358', '270', '273']
['Notes Receivable', 'NA', '0', '0', '0', '0']
['Inventories', '449', '430', '315', '292', '330']
['Other Current Assets', '27', '29', '34', '34', '27']
['Total Current Assets', '1,535', '1,019', '825', '694', '703']
['Net Property & Equipment', '380', '386', '377', '363', '371']
['Investments & Advances', 'NA', '61', '38', '48', '58']
['Other Non-Current Assets', 'NA', '0', '0', '0', '0']
['Deferred Charges', 'NA', '0', '0', '0', '0']
['Intangibles', 'NA', '233', '245', '213', '212']
['Deposits & Other Assets', '305', '7', '14', '25', '26']
['Total Assets', '2,242', '1,729', '1,529', '1,371', '1,370']
[]
['Notes Payable', 'NA', '0', '0', '0', '0']
['Accounts Payable', '128', '181',

Patrimônio Líquido (PL)

In [34]:
# Encontrando o valor de Total Shareholder's Equity
total_shareholders_equity = soup.find('td', text="Total Shareholder's Equity")
value = total_shareholders_equity.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
value_formatted = '${:,.2f}'.format(value_in_millions)

# Imprimindo o valor formatado
print(value_formatted)


$1,814,000,000.00


Empréstimos e Financiamentos

In [35]:
# Encontrando o valor de Current Portion Long-Term Debt
current_portion_longterm_debt = soup.find('td', text="Current Portion Long-Term Debt")
value = current_portion_longterm_debt.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions)

# Imprimindo o valor formatado
print('Empréstimos e Financiamentos:', value_formatted)

Empréstimos e Financiamentos: $1,000,000.00


Dívida de Longo Prazo

In [36]:
# Encontrando o valor de Long-Term Debt
longterm_debt = soup.find('td', text="Long-Term Debt")
value = longterm_debt.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions)

# Imprimindo o valor formatado
print('Dívida de Longo Prazo:', value_formatted)

Dívida de Longo Prazo: $1,000,000.00


Capital de Terceiros (Po)

In [37]:
# Encontrando o valor de Current Portion Long-Term Debt
current_portion_longterm_debt = soup.find('td', text="Current Portion Long-Term Debt")
value = current_portion_longterm_debt.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
current_portion_longterm_debt_in_millions = float(value.replace(',', '')) * 1000000

# Encontrando o valor de Long-Term Debt
longterm_debt = soup.find('td', text="Long-Term Debt")
value = longterm_debt.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
longterm_debt_in_millions = float(value.replace(',', '')) * 1000000

# Somando os valores de Current Portion Long-Term Debt e Long-Term Debt
total_debt = current_portion_longterm_debt_in_millions + longterm_debt_in_millions

# Formatando o valor de volta para uma string com a notação de dólares
if total_debt == 0:
  total_debt_formatted = '$ 0.00'
else:
  total_debt_formatted = '${:,.2f}'.format(total_debt)

# Imprimindo o valor formatado
print('Capital de Terceiros (PO):', total_debt_formatted)


Capital de Terceiros (PO): $2,000,000.00


Capital Investido (Ativos Totais)

In [38]:
# Encontrando o valor de Total Assets
total_assets = soup.find('td', text="Total Assets")
value = total_assets.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions)

# Imprimindo o valor formatado
print('Capital Investido (Ativos Totais):', value_formatted)

Capital Investido (Ativos Totais): $2,242,000,000.00


Caixa e Aplicações Financeiras

In [39]:
# Encontrando o valor de Cash & Equivalents
cash_equivalents = soup.find('td', text="Cash & Equivalents")
value = cash_equivalents.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions)

# Imprimindo o valor formatado
print('Caixa e Aplicações Financeiras:', value_formatted)

Caixa e Aplicações Financeiras: $679,000,000.00


Ativo Circulante

In [40]:
# Encontrando o valor de Total Current Assets
total_current_assets = soup.find('td', text="Total Current Assets")
value = total_current_assets.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions)

# Imprimindo o valor formatado
print('Ativo Circulante:', value_formatted)

Ativo Circulante: $1,535,000,000.00


Ativo Circulante Operacional

In [41]:
# Encontrando o valor de Total Current Assets
total_current_assets = soup.find('td', text="Total Current Assets")
value_assets = total_current_assets.find_next('td').text

# Encontrando o valor de Cash & Equivalents
cash_equivalents = soup.find('td', text="Cash & Equivalents")
value_cash = cash_equivalents.find_next('td').text

# Removendo as vírgulas e convertendo os valores para float
value_assets_in_millions = float(value_assets.replace(',', '')) * 1000000
value_cash_in_millions = float(value_cash.replace(',', '')) * 1000000

# Calcular o Ativo Circulante Operacional
value_operational = value_assets_in_millions - value_cash_in_millions

# Formatando o valor de volta para uma string com a notação de dólares
if value_operational == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_operational)

# Imprimindo o valor formatado
print('Ativo Circulante Operacional:', value_formatted)

Ativo Circulante Operacional: $856,000,000.00


Passivo Circulante

In [42]:
# Encontrando o valor de Total Current Liabilities
total_current_liabilities = soup.find('td', text="Total Current Liabilities")
value = total_current_liabilities.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions)

# Imprimindo o valor formatado
print('Passivo Circulante:', value_formatted)

Passivo Circulante: $348,000,000.00


Passivo Circulante Operacional

In [43]:
# Encontrando o valor de Total Current Liabilities
total_current_liabilities = soup.find('td', text="Total Current Liabilities")
value_liabilities = total_current_liabilities.find_next('td').text

# Encontrando o valor de Current Portion Long-Term Debt
current_portion_longterm_debt = soup.find('td', text="Current Portion Long-Term Debt")
value_debt = current_portion_longterm_debt.find_next('td').text

# Removendo as vírgulas e convertendo os valores para float
value_liabilities_in_millions = float(value_liabilities.replace(',', '')) * 1000000
value_debt_in_millions = float(value_debt.replace(',', '')) * 1000000

# Calcular o Passivo Circulante Operacional
value_operational = value_liabilities_in_millions - value_debt_in_millions

# Formatando o valor de volta para uma string com a notação de dólares
if value_operational == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_operational)

# Imprimindo o valor formatado
print('Passivo Circulante Operacional:', value_formatted)

Passivo Circulante Operacional: $347,000,000.00


## Cash Flow Statements

In [44]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.zacks.com/stock/quote/'+ symbol +'/cash-flow-statements'
user_agent = {'User-agent': 'Mozilla/5.0'}

# Obtendo a página
page = requests.get(url, headers = user_agent)

# Criando o objeto BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

# Extraindo os valores da tabela
table_rows = soup.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    print(row)

[]
['Strong Buy', '24.28%']
['Buy', '17.83%']
['Hold', '9.19%']
['Sell', '5.00%']
['Strong Sell', '2.16%']
['500', '10.51%']
['A']
['A']
['A']
['A']
[]
['Cash Flow From Operations, Investments & Financial Activities', '\xa0', '\xa0', '\xa0', '\xa0', '\xa0']
['Net Income (Loss)', '662.82', '475.12', '143.65', '106.23', '106.82']
['Depreciation/Amortization & Depletion', '44.09', '45.66', '45.16', '43.01', '39.87']
['Net Change from Assets/Liabilities', '17.63', '-177.66', '24.77', '15.73', '1.98']
['Net Cash from Discontinued Operations', 'NA', '0.00', '0.00', '0.00', '0.00']
['Other Operating Activities', '-0.59', '-31.42', '31.50', '35.57', '19.22']
['Net Cash From Operating Activities', '723.94', '311.70', '245.07', '200.54', '167.89']
['Property & Equipment', '-29.79', '-29.53', '-43.70', '-27.92', '-19.78']
['Acquisition/ Disposition of Subsidiaries', 'NA', '51.68', '-72.65', '3.47', '-167.68']
['Investments', '-217.86', '-1.61', '0.00', '-16.00', '-1.61']
['Other Investing Activit

Despesa Financeira

In [45]:
# Encontrando o valor de Issuance (Repayment) of Debt
issuance_repayment_of_debt = soup.find('td', text="Issuance (Repayment) of Debt")
value = issuance_repayment_of_debt.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions)

# Imprimindo o valor formatado
print('Despesa Financeira:', value_formatted)

Despesa Financeira: $-140,000.00


Depreciação e Amortização

In [46]:
# Encontrando o valor de Depreciação e Amortização
Depreciation_Amortization_Depletion = soup.find('td', text="Depreciation/Amortization & Depletion")
value = Depreciation_Amortization_Depletion.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions_3 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions_3 == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions_3)

# Imprimindo o valor formatado
print('Depreciação e Amortização:', value_formatted)

Depreciação e Amortização: $44,090,000.00


## DRE

In [47]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.zacks.com/stock/quote/'+ symbol +'/income-statement'
user_agent = {'User-agent': 'Mozilla/5.0'}

# Obtendo a página
page = requests.get(url, headers = user_agent)

# Criando o objeto BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

# Extraindo os valores da tabela
table_rows = soup.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    print(row)

[]
['Strong Buy', '24.28%']
['Buy', '17.83%']
['Hold', '9.19%']
['Sell', '5.00%']
['Strong Sell', '2.16%']
['500', '10.51%']
['A']
['A']
['A']
['A']
[]
['Sales', '3,982', '3,769', '2,398', '2,431', '2,508']
['Cost Of Goods', '2,865', '2,939', '1,966', '2,036', '2,150']
['Gross Profit', '1,118', '830', '432', '395', '357']
['Selling & Adminstrative & Depr. & Amort Expenses', '240', '175', '186', '204', '185']
['Income After Depreciation & Amortization', '877', '656', '246', '191', '173']
['Non-Operating Income', '0', '-7', '-17', '0', '3']
['Interest Expense', '1', '8', '19', '26', '25']
['Pretax Income', '876', '641', '209', '166', '150']
['Income Taxes', '223', '166', '55', '35', '31']
['Minority Interest', '5', '7', '4', '5', '2']
['Investment Gains/Losses', '10', '0', '-10', '-25', '-13']
['Other Income/Charges', 'NA', '0', '0', '0', '0']
['Income From Cont. Operations', '663', '475', '144', '106', '107']
['Extras & Discontinued Operations', '0', '0', '0', '0', '0']
['Net Income (GA

Receita

In [48]:
# Encontrando o valor de Sales
Sales = soup.find('td', text="Sales")
value = Sales.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions)

# Imprimindo o valor formatado
print('Receita:', value_formatted)

Receita: $3,982,000,000.00


Custos

In [49]:
# Encontrando o valor de Cost of Goods
Cost_of_Goods = soup.find('td', text="Cost Of Goods")
value = Cost_of_Goods.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions_1 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions_1 == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions_1)

# Imprimindo o valor formatado
print('Custos:', value_formatted)

Custos: $2,865,000,000.00


Resultado Bruto

In [50]:
# Encontrando o valor de Resultado Bruto
Resultado_Bruto = value_in_millions - value_in_millions_1
Resultado_Bruto 

if Resultado_Bruto == 0:
  Resultado_Bruto_formatted = '$ 0.00'
else:
  Resultado_Bruto_formatted = '${:,.2f}'.format(Resultado_Bruto)

# Imprimindo o valor formatado
print('Resultado Bruto:', Resultado_Bruto_formatted)

Resultado Bruto: $1,117,000,000.00


**EBIT**

In [51]:
# Encontrando o valor de Pretax Income
Pretax_Income = soup.find('td', text="Pretax Income")
value = Pretax_Income.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions_2 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions_2 == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions_2)

# Imprimindo o valor formatado
print('EBIT:', value_formatted)

EBIT: $876,000,000.00


IR

In [52]:
# Encontrando o valor de Income Taxes
Income_Taxes = soup.find('td', text="Income Taxes")
value = Income_Taxes.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions_3 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions_3 == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions_3)

# Imprimindo o valor formatado
print('IR:', value_formatted)

IR: $223,000,000.00


# *NOPAT Restrito* 

In [53]:
# Encontrando o valor de NOPAT Restrito
NOPAT_Restrito = value_in_millions_2 - value_in_millions_3
NOPAT_Restrito 

if NOPAT_Restrito == 0:
  NOPAT_Restrito_formatted = '$ 0.00'
else:
  NOPAT_Restrito_formatted = '${:,.2f}'.format(Resultado_Bruto)

# Imprimindo o valor formatado
print('NOPAT Restrito:', NOPAT_Restrito_formatted)

NOPAT Restrito: $1,117,000,000.00


Depesa Financeira

In [54]:
# Encontrando o valor de Investment Gains/Losses
Investment_Gains_Losses = soup.find('td', text="Investment Gains/Losses")
value = Investment_Gains_Losses.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions_4 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions_4 == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions_4)

# Imprimindo o valor formatado
print('Despesa Financeira:', value_formatted)

Despesa Financeira: $10,000,000.00


Lucro Líquido

In [55]:
# Encontrando o valor de Net Income (GAAP)
Net_Income_GAAP = soup.find('td', text="Net Income (GAAP)")
value = Net_Income_GAAP.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions_5 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions_5 == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions_5)

# Imprimindo o valor formatado
print('Lucro Líquido:', value_formatted)

Lucro Líquido: $658,000,000.00


## **CAPM - Custo do Capital Próprio**

Custo de Capital Próprio

In [56]:
rf = 4.5/100
rj = rf + (beta*(rm_perc - rf))
rj_str = str(rj).replace('.', ',')

print(f'Rj (Ke): {rj_str}')

Rj (Ke): 0,14700960000000002


Ki (Kd) - Custo da Dívida

PL / (Po+PL)

Po / (Po+PL)

**WACC**